In [ ]:
from openai import OpenAI
import numpy as np

# Initialize the OpenAI API client
api_key =  # Replace with your actual OpenAI API key

client = OpenAI(api_key=api_key)

In [ ]:
class Goal(BaseModel):
    description: str


class Goals(BaseModel):
    low_priority: list[Goal]
    medium_priority: list[Goal]
    high_priority: list[Goal]


gpt_sys_prompt = "\nYou are a data science master's student from Tampa, FL who just completed their 2-year program at the University of Pennsylvania in Philadelphia, PA. You are currently working on an extension of your capstone project, which is to develop a new AI agent that can play the board game Survivor."

gpt_user_prompt = "\nUsing the context above that describes the world and yourself, create high level goals at several priority levels for the next few months of your life. Make no more than 10 goals in total. Keep the goals concise."

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "system", "content": gpt_sys_prompt},
        {"role": "user", "content": gpt_user_prompt},
    ],
    response_format=Goals,
)

goals = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if goals.refusal:
    print(goals.refusal)
else:
    print(goals.parsed)

In [ ]:
goals.parsed

In [ ]:
import json

In [ ]:
goals_dict = goals.parsed.model_dump()  # Directly access the parsed dictionary
goals_dict

In [ ]:
type(goals_dict)

In [ ]:
[goal.description for goal in goals.parsed.low_priority]

In [ ]:
[goal.description for goal in goals.parsed.medium_priority]

In [ ]:
[goal.description for goal in goals.parsed.high_priority]

In [ ]:
completion

In [ ]:
goals

In [ ]:
# Example usage:
text1 = "company"
text2 = "corporation"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
node_id = 0
low_priority_goals = {
    node_id + i: goal.description for i, goal in enumerate(goals.parsed.low_priority)
}
node_id += len(goals.parsed.low_priority)

medium_priority_goals = {
    node_id + i: goal.description for i, goal in enumerate(goals.parsed.medium_priority)
}
node_id += len(goals.parsed.medium_priority)

high_priority_goals = {
    node_id + i: goal.description for i, goal in enumerate(goals.parsed.high_priority)
}

In [ ]:
low_priority_goals

In [ ]:
medium_priority_goals

In [ ]:
high_priority_goals

In [ ]:
gpt_user_prompt = (
    "Your Goals are as follows:\n"
    "Low Priority:\n\t"
    + "\n\t".join(
        [f"Node ID: {key}, Goal: {value}" for key, value in low_priority_goals.items()]
    )
    + "\n\n"
    "Medium Priority:\n\t"
    + "\n\t".join(
        [
            f"Node ID: {key}, Goal: {value}"
            for key, value in medium_priority_goals.items()
        ]
    )
    + "\n\n"
    "High Priority:\n\t"
    + "\n\t".join(
        [f"Node ID: {key}, Goal: {value}" for key, value in high_priority_goals.items()]
    )
    + "\n\n"
    "Using the context above that describes the world and yourself, assign to each of the goals a progress score between 0 and 10 based on how important it is that you make progress on the goal. The scores should be in order of priority, with the highest priority goals having the highest scores."
)

print(gpt_user_prompt)

In [ ]:
class Score(BaseModel):
    node_id: int
    progress_score: int


class Scores(BaseModel):
    scores: list[Score]


gpt_sys_prompt = "\nYou are a data science master's student from Tampa, FL who just completed their 2-year program at the University of Pennsylvania in Philadelphia, PA. You are currently working on an extension of your capstone project, which is to develop a new AI agent that can play the board game Survivor."

gpt_user_prompt = (
    "Your Goals are as follows:\n"
    "Low Priority:\n\t"
    + "\n\t".join(
        [f"Node ID: {key}, Goal: {value}" for key, value in low_priority_goals.items()]
    )
    + "\n\n"
    "Medium Priority:\n\t"
    + "\n\t".join(
        [
            f"Node ID: {key}, Goal: {value}"
            for key, value in medium_priority_goals.items()
        ]
    )
    + "\n\n"
    "High Priority:\n\t"
    + "\n\t".join(
        [f"Node ID: {key}, Goal: {value}" for key, value in high_priority_goals.items()]
    )
    + "\n\n"
    "Using the context above that describes the world and yourself, assign to each of the goals a progress score between 0 and 10 based on how important it is that you make progress on the goal. The scores should be in order of priority, with the highest priority goals having the highest scores."
)

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "system", "content": gpt_sys_prompt},
        {"role": "user", "content": gpt_user_prompt},
    ],
    response_format=Scores,
)

scores = completion.choices[0].message.parsed

In [ ]:
scores

In [ ]:
{score.node_id: score.progress_score for score in scores.scores}

In [ ]:
%pip install dill
%pip install torch
%pip install bert
%pip install transformers

In [ ]:
# importing libraries
from transformers import RobertaTokenizer, RobertaModel
import torch
from torch.nn.functional import cosine_similarity

In [ ]:
# from transformers import RobertaTokenizer, RobertaModel
# from torch.nn.functional import cosine_similarity
# import torch
# import logging


# # Set logging level for transformers to ERROR to suppress warnings
# logging.getLogger("transformers").setLevel(logging.ERROR)


# def cosine_similarity_contextual_embeddings(text1, word1, text2, word2, mode="average"):
#     """
#     Compute the cosine similarity between the contextual embeddings of two words in their respective texts.

#     Parameters:
#     - text1: First piece of text
#     - word1: Target word in the first text
#     - text2: Second piece of text
#     - word2: Target word in the second text
#     - mode: 'average' or 'each'.
#             'average' averages the embeddings for all occurrences of the word.
#             'each' returns cosine similarity for each occurrence of the word separately.

#     Returns:
#     - Cosine similarity (averaged or per instance) between the contextualized embeddings of the two words.
#     """

#     # Initialize the RoBERTa tokenizer and model
#     tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
#     model = RobertaModel.from_pretrained("roberta-base")

#     def get_word_embeddings(text, word):
#         # Tokenize the input text
#         inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)
#         tokenized_text = tokenizer.tokenize(text)

#         # Tokenize the target word
#         word_tokens = tokenizer.tokenize(word)
#         if word_tokens[0][0] != "Ġ":
#             # Add the 'Ġ' if not present, since RoBERTa uses it to indicate a space before the word
#             word_tokens = ["Ġ" + word_tokens[0]] + word_tokens[1:]

#         # Get RoBERTa embeddings for the entire text
#         with torch.no_grad():
#             outputs = model(**inputs)
#             last_hidden_state = outputs.last_hidden_state

#         # Find positions of the word in the tokenized input by matching subword tokens
#         word_positions = []
#         for i in range(len(tokenized_text) - len(word_tokens) + 1):
#             # Check if the sequence of tokens in the text matches the word's token sequence
#             if tokenized_text[i : i + len(word_tokens)] == word_tokens:
#                 word_positions.append(i)  # Start index of the matched subwords

#         if not word_positions:
#             raise ValueError(
#                 f"Word '{word}' not found in the input text. Tokenized text: {tokenized_text}"
#             )

#         # Get embeddings for each occurrence of the word by averaging subword embeddings
#         word_embeddings = []
#         for pos in word_positions:
#             # Get embeddings for all subword tokens of the word
#             subword_embeddings = last_hidden_state[:, pos : pos + len(word_tokens), :]
#             word_embedding = subword_embeddings.mean(
#                 dim=1
#             )  # Average subword embeddings
#             word_embeddings.append(word_embedding)

#         word_embeddings = torch.stack(word_embeddings, dim=0)
#         return word_embeddings

#     # Get embeddings for both words in their respective contexts
#     embeddings1 = get_word_embeddings(text1, word1)
#     embeddings2 = get_word_embeddings(text2, word2)

#     if mode == "average":
#         # Average all occurrences
#         embedding1 = embeddings1.mean(dim=0)
#         embedding2 = embeddings2.mean(dim=0)
#         # Compute cosine similarity between the averaged embeddings
#         similarity = cosine_similarity(embedding1, embedding2)
#         return similarity.item()

#     elif mode == "each":
#         # Compute cosine similarity for each occurrence of word1 with each occurrence of word2
#         similarities = []
#         for emb1 in embeddings1:  # Get individual embeddings of word1
#             for emb2 in embeddings2:  # Get individual embeddings of word2
#                 similarity = cosine_similarity(emb1, emb2)
#                 similarities.append(similarity.item())
#         return similarities

#     else:
#         raise ValueError("Invalid mode. Choose 'average' or 'each'.")


# # Example usage
# text1 = "The bank of the river was peaceful, but he also visited the bank to withdraw money."
# word1 = "bank"

# text2 = "He stood at the riverbank and watched the water flow gently by."
# word2 = "riverbank"

# # Averaged cosine similarity
# similarity_score = cosine_similarity_contextual_embeddings(
#     text1, word1, text2, word2, mode="average"
# )
# print(f"Averaged cosine similarity between '{word1}' and '{word2}': {similarity_score}")

# # Cosine similarity for each occurrence
# similarity_scores_each = cosine_similarity_contextual_embeddings(
#     text1, word1, text2, word2, mode="each"
# )
# print(
#     f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
# )

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity


def cosine_similarity_contextual_embeddings(text1, word1, text2, word2, mode="average"):
    """
    Compute the cosine similarity between the contextual embeddings of two words in their respective texts.

    Parameters:
    - text1: First piece of text
    - word1: Target word in the first text
    - text2: Second piece of text
    - word2: Target word in the second text
    - mode: 'average' or 'each'.
            'average' averages the embeddings for all occurrences of the word.
            'each' returns cosine similarity for each occurrence of the word separately.

    Returns:
    - Cosine similarity (averaged or per instance) between the contextualized embeddings of the two words.
    """

    # Initialize the BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    def get_word_embeddings(text, word):
        # Tokenize the input text
        inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True)
        word_tokens = tokenizer.tokenize(word)

        # Get input_ids for the target word
        word_token_ids = tokenizer.convert_tokens_to_ids(word_tokens)

        # Get BERT embeddings for the entire text
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state[0]  # Remove batch dimension

        # Find positions of the word in the tokenized input
        word_positions = [
            i
            for i, token_id in enumerate(inputs["input_ids"][0])
            if token_id in word_token_ids
        ]

        # Get embeddings for each occurrence of the word
        word_embeddings = last_hidden_state[word_positions, :]

        return word_embeddings

    # Get embeddings for both words in their respective contexts
    embeddings1 = get_word_embeddings(text1, word1)
    embeddings2 = get_word_embeddings(text2, word2)

    if mode == "average":
        # Average all occurrences
        embedding1 = embeddings1.mean(dim=0)
        embedding2 = embeddings2.mean(dim=0)
        # Compute cosine similarity between the averaged embeddings
        similarity = cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0))
        return similarity.item()

    elif mode == "each":
        # Compute cosine similarity for each occurrence of word1 with each occurrence of word2
        similarities = []
        for emb1 in embeddings1:  # Get individual embeddings of word1
            for emb2 in embeddings2:  # Get individual embeddings of word2
                similarity = cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0))
                similarities.append(similarity.item())
        return similarities

    else:
        raise ValueError("Invalid mode. Choose 'average' or 'each'.")


# Example usage
text1 = "The bank of the river was peaceful, but he also visited the bank to withdraw money."
word1 = "bank"

text2 = "He stood at the riverbank and watched the water flow gently by."
word2 = "riverbank"

# Averaged cosine similarity
similarity_score = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="average"
)
print(f"Averaged cosine similarity between '{word1}' and '{word2}': {similarity_score}")

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "The corporation is a large company that makes money."
word1 = "corporation"

text2 = "The company has seen year-over-year and quarter-over-quarter growth since its inception."
word2 = "company"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "cat"
word1 = "cat"

text2 = "kitten"
word2 = "kitten"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "The cat is a small animal."
word1 = "cat"

text2 = "The kitten is a small animal."
word2 = "kitten"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "The cat played with the ball."
word1 = "cat"

text2 = "The kitten is a small animal."
word2 = "kitten"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "The baby went to the park."
word1 = "baby"

text2 = "The kitten is stretching on the rug."
word2 = "kitten"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "baby"
word1 = "baby"

text2 = "kitten"
word2 = "kitten"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "corporation"
word1 = "corporation"

text2 = "company"
word2 = "company"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "firm"
word1 = "firm"

text2 = "company"
word2 = "company"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
# Example usage
text1 = "corporation"
word1 = "corporation"

text2 = "firm"
word2 = "firm"

# Cosine similarity for each occurrence
similarity_scores_each = cosine_similarity_contextual_embeddings(
    text1, word1, text2, word2, mode="each"
)
print(
    f"Cosine similarity between '{word1}' and '{word2}' for each occurrence: {similarity_scores_each}"
)

In [ ]:
from openai import OpenAI
import numpy as np

# Initialize the OpenAI API client
api_key = None  # Replace with your actual OpenAI API key

client = OpenAI(api_key=api_key)


def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


# def compare_texts(text1, text2):
# """Compare two pieces of text and return their cosine similarity score."""
# embeddings = get_embeddings([text1, text2])
# return cosine_similarity(embeddings)

In [ ]:
# Example usage:
text1 = "baby"
text2 = "kitten"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "cat"
text2 = "kitten"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "mouse"
text2 = "mice"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "cat"
text2 = "dog"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "company"
text2 = "zoo"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "The cat is a small animal."
text2 = "The kitten is a small animal."
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "company"
text2 = "corporation"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "firm"
text2 = "corporation"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "firm"
text2 = "company"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
text1 = "The corporation is a large company that makes money."
text2 = "The company has seen year-over-year and quarter-over-quarter growth since its inception."

text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "business"
text2 = "company"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")

In [ ]:
# Example usage:
text1 = "business"
text2 = "company"
text1_embedding = get_embedding(text1)
text2_embedding = get_embedding(text2)

similarity_score = cosine_similarity(
    torch.tensor(text1_embedding).unsqueeze(0),
    torch.tensor(text2_embedding).unsqueeze(0),
)
print(f"Cosine Similarity Score: {similarity_score.item():.4f}")